In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM
import torch

In [32]:
import os 

print(os.getcwd())

/work/qvr905


In [33]:

model_id = "./FineTune/mistral/mistral_code"

# Load Model with PEFT adapter
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  load_in_4bit=True,
  device_map="auto",
  cache_dir="./.cache/huggingface"
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.97s/it]


In [34]:
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [58]:
def generate_response(model, tokenizer, query, context=None):
    # Ensure that the model and input tensors are on the same device
    device = next(model.parameters()).device
    
    # Tokenize inputs
    inputs = tokenizer(query, context, return_tensors="pt", add_special_tokens=True)
    # Set pad token ID and padding side
    tokenizer.padding_side = 'left'
    input_ids = inputs["input_ids"].to(device)
    
    # Generate output
    output = model.generate(input_ids=input_ids,max_new_tokens =256 ,temperature = 0.9,top_k=100,num_beams=20)
    #output = model.generate(input_ids=input_ids,max_new_tokens =256)
    
    
    # Decode and return response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [36]:
from tqdm import tqdm
import evaluate
from codebleu import calc_codebleu
from bert_score import score as bert_score

# Load evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
bert_metric = evaluate.load("bertscore")
codebleu_metric = evaluate.load("k4black/codebleu")

def evaluate(predictions, references,model):    
    # Compute BERTScore
    #bert_score_precision, bert_score_recall, bert_score_f1= bert_metric.compute(predictions=predictions, references=references, lang='en', verbose=False)
    bert_score_precision, bert_score_recall, bert_score_f1 = bert_score(predictions, references, lang="en", verbose=False)
    #bert_score = bert_score_result['bertscore'].item()
    # Compute BLEU
    bleu_score = bleu_metric.compute(predictions=predictions, references=references,smooth=True)['bleu']
    
    # Compute Rouge-L
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)['rougeL']
    
    codebleu = calc_codebleu(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
    return bleu_score, rouge_score,codebleu, bert_score_precision, bert_score_recall, bert_score_f1


In [37]:
from datasets import load_dataset
from random import randrange


dataset = load_dataset("flytech/python-codes-25k", split = "train", cache_dir="./.cache/huggingface")
dataset = dataset.train_test_split(test_size=0.1)
#dataset = dataset.map(formatting_prompts_func, batched = True,)
print(len(dataset["train"]))
print(len(dataset["test"]))

for i in range(5):
    print(dataset["train"][i])

print(f"dataset size: {len(dataset['test'])}")

44663
4963
{'output': '```python\ndef staircase(n): \n    for i in range(1, n + 1): \n        for j in range(1, n + 1): \n            if (j <= (n - i)):   \n                print(end="\\t") \n            else: \n                print("#\\t", end="") \n        print()\n```', 'text': 'Write a Python function that prints out a staircase pattern. No problem, I\'m on it! ```python\ndef staircase(n): \n    for i in range(1, n + 1): \n        for j in range(1, n + 1): \n            if (j <= (n - i)):   \n                print(end="\\t") \n            else: \n                print("#\\t", end="") \n        print()\n```', 'instruction': 'Write a Python function that prints out a staircase pattern.', 'input': ''}
{'output': "```python\nimport xml.etree.ElementTree as ET\n\n# Let me run it for you!\n\ntree = ET.parse('file.xml')\nroot = tree.getroot()\nfor child in root:\n    print(child.tag, child.attrib)\n```", 'text': "Write Python code to read an XML file and parse it using ElementTree. Readi

In [3]:
# from datasets import load_dataset
# from random import randint

# from datasets import load_dataset
# eval_dataset = load_dataset("sg247/python-codes-25k-llama2", split = "train")
# eval_dataset = eval_dataset.train_test_split(test_size=0.1, shuffle=True)
# #dataset = dataset.map(formatting_prompts_func, batched = True,)
# print(len(eval_dataset["train"]))
# print(len(eval_dataset["test"]))
# rand_idx = randint(0, len(eval_dataset))

# for i in range(5):
#     print(eval_dataset["train"][i])

Generating train split: 100%|██████████| 49626/49626 [00:00<00:00, 163404.56 examples/s]


44663
4963
{'output': "```python\nfrom bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser'))\n# Code executed.\n```", 'text': "Execute code: from bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser')) Parsing HTML with BeautifulSoup... ```python\nfrom bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser'))\n# Code executed.\n```", 'input': '', 'instruction': "Execute code: from bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser'))", 'formatted_text': "<s> [INST] Execute code: from bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser')) [/INST] ```python\nfrom bs4 import BeautifulSoup\nprint(BeautifulSoup('<p>Hello</p>', 'html.parser'))\n# Code executed.\n``` </s>"}
{'output': '```python\nimport eli5\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.datasets import make_classification\n\n# Creating a dataset\nX, y = make_classification()\n\n# Fitting the mo

In [38]:
eval_dataset = dataset["test"]

In [59]:
import re
# Initialize lists to store results
pred = []
iter = 0
gt =[]
# Iterate through the queries
number_of_eval_samples = 50

dataseteval = eval_dataset.select(range(number_of_eval_samples))
for query in dataseteval:  
    instruction = query["instruction"]
    ground = query["output"]
    gt.append(query["output"])
        
    input_prompt = f"{instruction}"
    respons = generate_response(model, tokenizer, input_prompt)
    #print(respons)
    generated_text = respons
    pred.append(generated_text)

    # Print the generated text
    print(f"Instruction:{instruction}")
    print(f"Ground truth output:\n{ground}")    
    print(f"Generated Answer:\n{generated_text}")
    print(f"sample:{iter}")
    iter+=1

/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Instruction:Generate a Python script for setting up a basic web server for hosting static content
Ground truth output:
```python
#!/usr/bin/env python


import http.server

# Set the port number
port_number = 8080

# Create the HTTP server
server_address = ('', port_number)
httpd = http.server.HTTPServer(server_address, http.server.SimpleHTTPRequestHandler)

print('Server started on port {}'.format(str(port_number)))

# Run the HTTP server
httpd.serve_forever()
```
Generated Answer:
Generate a Python script for setting up a basic web server for hosting static content Generate a Python script for setting up a basic web server for hosting static content... ```python
import http.server
import socketserver
PORT = 8000
Handler = http.server.SimpleHTTPRequestHandler
with socketserver.TCPServer(('', PORT), Handler) as httpd:
    print(f'Serving at port {PORT}')
    httpd.serve_forever()
```
```python
import http.server
import socketserver
PORT = 8000
Handler = http.server.SimpleHTTPRequestHan

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Instruction:Fix the code: import torch
import torch.nn.functional as F
input_data = torch.tensor([1.0, 0.5])
weights = torch.tensor([0.1, 0.2])
output = F.linear(input_data, weights)
Ground truth output:
```python
import torch
import torch.nn.functional as F

# Creating input data and weights tensors
input_data = torch.tensor([1.0, 0.5])
weights = torch.tensor([0.1, 0.2])

# Performing linear transformation
output = F.linear(input_data, weights)

# Let me examine if the code works
# 1. Performed a linear transformation using functional API
# Executing code...
# Code has been fixed!
```
Generated Answer:
Fix the code: import torch
import torch.nn.functional as F
input_data = torch.tensor([1.0, 0.5])
weights = torch.tensor([0.1, 0.2])
output = F.linear(input_data, weights) Fixing the code by performing a linear transformation using PyTorch... ```python
import torch
import torch.nn.functional as F

input_data = torch.tensor([1.0, 0.5])
weights = torch.tensor([0.1, 0.2])
output = F.linear(

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Instruction:List out all the sorting algorithms supported in Python.
Ground truth output:
The sorting algorithms supported in Python include bubble sort, insertion sort, selection sort, merge sort, quick sort, and heap sort.
Generated Answer:
List out all the sorting algorithms supported in Python. Listing out all the sorting algorithms supported in Python... ```python
sorting_algorithms = ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```
### Output: ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```python
sorting_algorithms = ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```
### Output: ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```python
sorting_algorithms = ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```
### Output: ['bubble_sort', 'insertion_sort', 'selection_sort', 'merge_sort', 'quick_sort']
```python
sorting

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Instruction:Write a Python program to compute the sum of the absolute difference of all pairs of numbers in a given array [5, 8, 10, 15]
Ground truth output:
```python
arr = [5, 8, 10, 15]
Sum = 0

for i in range(len(arr)):
    for j in range(len(arr)):
        if(i != j):
            Sum += abs(arr[i] - arr[j])
            
print(Sum)
```
Generated Answer:
Write a Python program to compute the sum of the absolute difference of all pairs of numbers in a given array [5, 8, 10, 15] Writeing a Python program to compute the sum of the absolute difference of all pairs of numbers in a given array... ```python
arr = [5, 8, 10, 15]
sum = 0
for i in range(len(arr)):
    for j in range(i + 1, len(arr)):
        sum += abs(arr[i] - arr[j])
print(sum)
```
# Output: 30
```python
arr = [5, 8, 10, 15]
sum = 0
for i in range(len(arr)):
    for j in range(i + 1, len(arr)):
        sum += abs(arr[i] - arr[j])
print(sum)
```
# Output: 30
```python
arr = [5, 8, 10, 15]
sum = 0
for i in range(len(arr)):
  

In [15]:
# Evaluate metrics
bleu, rouge ,codebleu, bert_score_precision, bert_score_recall, bert_score_f1= evaluate(pred, gt,model)
print(str(bleu))
print(str(rouge))
print(str(codebleu))
print(bert_score_precision.mean().item(), bert_score_recall.mean().item(), bert_score_f1.mean().item())



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


0.013837160660285603
0.1090903436809324
{'codebleu': 0.13331116210210073, 'ngram_match_score': 0.013591339338793447, 'weighted_ngram_match_score': 0.08812219891198744, 'syntax_match_score': 0.10194730813287514, 'dataflow_match_score': 0.3295838020247469}
0.6861045956611633 0.7815998792648315 0.7299688458442688


In [ ]:
# # Initialize lists to store results
# import gc
# from tqdm import tqdm

# iter = 0
# gc.collect()
# torch.cuda.empty_cache()

# gt =[]
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# # Iterate through the queries
# number_of_eval_samples = 100
# dataseteval = eval_dataset.shuffle().select(range(number_of_eval_samples))
# instr=[]
# out = []
# for sample in dataseteval:
#     instruction = sample["instruction"]
#       # Define your prompt using the instruction variable
#     prompt = f"""<start_of_turn>{instruction}<end_of_turn>
#     <start_of_turn>model"""
#     input_prompt = f"<s>[INST] {prompt} [/INST]"
    
#     instr.append(input_prompt)
#     gt.append(sample['output'])
# print(len(instr))

# batch_size = 3
# for sample in tqdm(list(chunks(instr,batch_size))):
    
#     encoding = tokenizer(instr, return_tensors="pt",padding = True, truncation = True).to(device)
#     with torch.no_grad():
#         genereated_ids = model.generate(**encoding,max_new_tokens =256, temperature = 0.9)
#     generated_texts = tokenizer.batch_decode(genereated_ids, skip_special_tokens=True)

# for sample in dataseteval:
#     instruction = sample["instruction"]
#     ground = sample["output"]
    
#     # Print the generated text
#     print(f"Instruction:{instruction}")
#     print(f"Ground truth output:\n{ground}")    
#     print(f"Generated Answer:\n{generated_texts[iter]}")
#     iter+=1

In [4]:
# from tqdm import tqdm
# import evaluate
# from codebleu import calc_codebleu

# # Load evaluation metrics
# bleu_metric = evaluate.load("bleu")
# rouge_metric = evaluate.load("rouge")
# bert_metric = evaluate.load("bertscore")
# codebleu_metric = evaluate.load("k4black/codebleu")

In [ ]:
# eval = evaluate.evaluator("text-generation")
# result = eval.compute(
#     model_or_pipeline=pipe,
#     data=eval_dataset["test"],
#     metric=bleu_metric,
#     input_column="", 
#     label_column = ""
# )

# print(result)

In [15]:
# # Fetch instruction from eval_dataset
# instruction = eval_dataset['test'][rand_idx]["instruction"]
# groundtruth = eval_dataset['test'][rand_idx]["output"]

# # Define your prompt using the instruction variable
# prompt = f"""<start_of_turn>{instruction}<end_of_turn>
# <start_of_turn>model"""

# # Use the pipeline for text generation
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
# generated_text = result[0]['generated_text']
# generated_code = generated_text.split("```python")[1].split("```")[0].strip()
# # Print the generated text
# #print(result[0]['generated_text'])
# print(f"Instruction:{instruction}")
# print(f"Ground truth output:\n{groundtruth}")
# print(f"Generated Answer:\n{generated_code}")


Instruction:Utilize the Python in-built module to print the calendar for the given month Month: 5
Year: 2021
Ground truth output:
```python
# import the module
import calendar

# print the calendar 
print(calendar.month(2021, 5))
```
Generated Answer:
import calendar
c = calendar.TextCalendar(calendar.SUNDAY)
str(c.formatmonth(2021, 5))


In [20]:


# # Initialize lists to store results
# responses = []
# iter = 0
# # Iterate through the queries
# number_of_eval_samples = 5
# dataseteval = eval_dataset['test'].shuffle().select(range(number_of_eval_samples))
# for query in dataseteval:  
#     instruction = query["instruction"]
#     groundtruth = query["output"]
    
#     # Define your prompt using the instruction variable
#     prompt = f"""<start_of_turn>{instruction}<end_of_turn>
#     <start_of_turn>model"""
#     input_prompt = f"<s>[INST] {prompt} [/INST]"

#     # Generate response without RAG
#     input_prompt = f"question: {query}"
#     respons = generate_response(model, tokenizer, input_prompt)
    
    
#     responses.append(respons)
#     print("Question:" + input_prompt)
#     print("Response Llama: " + respons)
#     iter+=1
   


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:question: {'text': "Write a Python script which checks if an input string is a palindrome. Yes, let's get down to business! ```python\ndef is_palindrome(input_string):\n    input_string = input_string.lower()\n    if len(input_string) == 0:\n        return False\n    if len(input_string) == 1:\n        return True\n\n    if input_string[0] == input_string[-1]:\n        return is_palindrome(input_string[1:-1])\n\n    return False\n\n# Usage \nstring = 'RADAR'\nresult = is_palindrome(string)\nprint(result) # Output: True\n```", 'input': '', 'output': "```python\ndef is_palindrome(input_string):\n    input_string = input_string.lower()\n    if len(input_string) == 0:\n        return False\n    if len(input_string) == 1:\n        return True\n\n    if input_string[0] == input_string[-1]:\n        return is_palindrome(input_string[1:-1])\n\n    return False\n\n# Usage \nstring = 'RADAR'\nresult = is_palindrome(string)\nprint(result) # Output: True\n```", 'instruction': 'Write a Pyt

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:question: {'text': "Write a python script to compare two directories and output the differences dir1/\n        file1.txt\n        file2.txt\n\ndir2/\n        file1.txt\n        file3.txt Consider it in progress! Working on it now! ```python\nimport os\n\ndef compare_dirs(dir1, dir2):\n    dir1_listing = os.listdir(dir1)\n    dir2_listing = os.listdir(dir2)\n\n    dir1_files = set(dir1_listing)\n    dir2_files = set(dir2_listing)\n\n    new_files = dir2_files.difference(dir1_files)\n    deleted_files = dir1_files.difference(dir2_files)\n\n    print('NEW FILES:')\n    for f in new_files:\n        print(f)\n\n    print('DELETED FILES:')\n    for f in deleted_files:\n        print(f)\n\n\ncompare_dirs('dir1/', 'dir2/')\n```", 'input': '', 'output': "```python\nimport os\n\ndef compare_dirs(dir1, dir2):\n    dir1_listing = os.listdir(dir1)\n    dir2_listing = os.listdir(dir2)\n\n    dir1_files = set(dir1_listing)\n    dir2_files = set(dir2_listing)\n\n    new_files = dir2_files.dif

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:question: {'text': 'Can you write an Python program to find the sum of two given numbers? Firing up the engines! We\'re almost airborne! ```python\ndef sum_integers(a, b): \n    return a + b \n    \n# Driver Program \na = 4\nb = 5\n\nprint("Sum of {} and {} is: {}".format(a, b, sum_integers(a, b)))\n```', 'input': '', 'output': '```python\ndef sum_integers(a, b): \n    return a + b \n    \n# Driver Program \na = 4\nb = 5\n\nprint("Sum of {} and {} is: {}".format(a, b, sum_integers(a, b)))\n```', 'instruction': 'Can you write an Python program to find the sum of two given numbers?'}
Response Llama: question: {'text': 'Can you write an Python program to find the sum of two given numbers? Firing up the engines! We\'re almost airborne! ```python\ndef sum_integers(a, b): \n    return a + b \n    \n# Driver Program \na = 4\nb = 5\n\nprint("Sum of {} and {} is: {}".format(a, b, sum_integers(a, b)))\n```', 'input': '', 'output': '```python\ndef sum_integers(a, b): \n    return a + b \

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:question: {'text': "Run Disk Cleanup Running Disk Cleanup... ```python\nimport os\nos.system('cleanmgr')\n```", 'input': 'Running Disk Cleanup...', 'output': "```python\nimport os\nos.system('cleanmgr')\n```", 'instruction': 'Run Disk Cleanup'}
Response Llama: question: {'text': "Run Disk Cleanup Running Disk Cleanup... ```python\nimport os\nos.system('cleanmgr')\n```", 'input': 'Running Disk Cleanup...', 'output': "```python\nimport os\nos.system('cleanmgr')\n```", 'instruction': 'Run Disk Cleanup'} Write a Python code to run Disk Cleanup. ```python
import os
os.system('cleanmgr')
```
Running Disk Cleanup... ```python
import os
os.system('cleanmgr')
```
# Output: Running Disk Cleanup...
```python
import os
os.system('cleanmgr')
```
```python
import os
os.system('cleanmgr')
```
# Running Disk Cleanup...
```python
import os
os.system('cleanmgr')
```
```python
import os
os.system('cleanmgr')
```
# Running Disk Cleanup...
```python
import os
os.system('cleanmgr')
```
# Output: Ru

References: 
https://www.philschmid.de/fine-tune-llms-in-2024-with-trl